 ## HUMBIO51 Assignment Week 6

##  Learning Objectives
***Students should be able to***

### How to represent a transcription factor's binding motif and scan sequences for matches 

<ol> 
 <li> <a href=#PCA2>Perform prinicipal component analysis to identify clustering patterns in a dataset </a></li>
 <li> <a href=#Scatter>Make a scatter plot of the output from principal component analysis</a></li>
<li> <a href=#Introkmeans>Explain what the k-means clustering algorithm is </a></li>
<li> <a href=#Runkmeans>Perform k-means clustering to examine groups of samples from an experiment.</a></li>
<li> <a href=#heatmap>Interpret data in a heatmap </li>
<li>  <a href=#TFMotif>Explain what a transcription factor binding motif is</a> </li>
<li> <a href=#ChIPseq>Understand what is a ChIP-seq experiment and how it can be used to identify transcription factor binding genome-wide</a> </li>
<li> <a href=#ChipseqBrowser>Visualize transcription factor ChIP-seq experiment in the WashU Genome Browser</a> </li>
<li><a href=#Regulator> Explain what a regulatory element is </a></li>
<li> <a href=#PromotersEnhancers>Explain what promoters and enhancers are</a></li>
</ol> 

## Question 1: Using k-means analysis to cluster genomic regions by chromatin state 

We have sampled 10000 regions from the human reference genome at random. Each region is 200 base pairs long. At each of these regions, we have data from a CHIP-Seq experiment within the GM12878 cell (blood cell line) measuring the strength of 5 histone markers: 
* H3K4me3, 
* H3K4me1, 
* H3K36me3, 
* H3K9me3,
* H3K27me3. 

The data is stored in a file in your Week_6 folder called **region_x_chrom_mark.tsv**

This file contains 200 bp genomic regions along the y-axis and values of "0", "0.5", or "1" for each of the 5 histone marks along the x-axis. 0 indicates that the histone mark is not present, 0.5 indicates a weak presence, and 1 indicates a strong presence. 


In [ ]:
#First, let's import the helper functions you will need to complete the homework: 
import sys
sys.path.append('../helpers')
import plotly 
plotly.offline.init_notebook_mode()
import plotly.plotly as py
from plotly.graph_objs import * 

import plotly_helpers 
from plotly_helpers import * 
import kmeans_helpers 
from kmeans_helpers import * 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
import pandas as pd


**Question 1a. Perform principal component analysis on the data matrix in region_x_chrom_mark.tsv. Create a scatter plot to visualize PC1 along the x-axis and PC2 along the y-axis.**

Hint: refer to the PCA code from class 8.

Hint: We are performing PCA on the genome regions, which are located in the rows of the matrix. Therefore, there is not need to transpose the matrix before running PCA. 

In [ ]:
## Your code here ##


**Question 1b: Looking at the PCA plot, how many clusters would you expect to find in a K-means analysis of this data?**

Your answer here:  

**Question 1c: Use the k-means clustering algorithm implementation from class 9 to cluster the 10,000 genomic regions. Use values of k = 2,5,15, 20. Which value of K gives the best clustering as measured by the silhouette score ? ** 

In [ ]:
## Your code here -- 
## Remove the number signs to uncomment the code, and replace the question marks with correct values ##

#k=?
#clusters=scikit_kmeans(?,?,?,?,?)

**Quesion 1d. Use your list "clusters" from 1b. to plot a bar chart indicating how many genes are assigned to each cluster.**


In [ ]:
##  Use the helper function below to generate a dictionary with the counts of genome regions in each cluster. 
def tally_cluster_counts(clusters): 
    tally_dict=dict() 
    for c in clusters: 
        if c not in tally_dict: 
            tally_dict[c]=1 
        else: 
            tally_dict[c]+=1 
    return tally_dict 

#YOUR CODE HERE 
## Call the tally_cluster_counts function on your clusters, and store the output in a variable called "tally_dict"
#Remove the number signs to uncomment the code, and replace the question marks with correct values ##

#tally_dict=? 

#Plot a bar chart of the values in taly_dict, with keys along the x-axis and values along the y-axis. 
#Hint: Making a bar chart is very similar to the scatterplots you generated in class 8. 


#YOUR CODE HERE 

**Question 1e. Below, we have written a function that will plot a heatmap of your clusters. The x-axis shows the mean of the H3K4me3, H3K4me1, H3K36me3, H3K9me3, and H3K27me3 values for each cluster. The y-axis lists the clusters that you generated. Run this function.**

In [ ]:
import seaborn as sns
%matplotlib inline   
def plot_cluster_heatmap(data,clusters,k):
    '''
    k -- the value of k that gave the best clustering of the data. 
    data -- original data frame with genomic regions along the y-axis and histone marks along the x-axis 
    clusters -- list of cluster assignments from K-means clustering. 
    '''
    cluster_summary=None
    for c in range(k): 
        #extract the genomic regions assigned to the current cluster c. 
        cur_subset=data.iloc[np.where(clusters==c)]
        #calculate the mean value for each histone mark in cur_subset 
        mean_histone_vals=np.mean(cur_subset)
        if c==0: 
            cluster_summary=mean_histone_vals 
        else: 
            cluster_summary=pd.concat([cluster_summary,mean_histone_vals],axis=1,ignore_index=True)
    flatui = ["#FFFFFF", "#CCCCCC","#000000"]
    sns.heatmap(cluster_summary.transpose(),annot=False,cmap=sns.color_palette(flatui))


In [ ]:
##YOUR CODE HERE ##

The heatmap shows that each cluster corresponds to a distinct pattern of chromatin histone marks. The ChromHMM project has mapped these combinationsn of histone marks to the 15 distinct chromatin states included in the annotation file **15_state_chromHMM_annotations.tsv** : 

http://egg2.wustl.edu/roadmap/data/byFileType/chromhmmSegmentations/ChmmModels/coreMarks/jointModel/final/annotationEnrichment_RoadmapEp_coreMarks_15State.png.

Focus on the "Emissions" chart in the top left portion of the table: 


![ChromHMM 15 state Model](../Images/Weekly_HW_6_ChromHMM.png)

**Question 1f. Which cluster maps to which chromatin state? Create a dictionary called "annotation_dict" that maps each cluster id to a ChromHMM annotation state.** 


In [ ]:
annotation_dict=dict() 
#for example: 
annotation_dict[0]="Heterochromatin"
#Fill in the other cluster - state mappings. 

## Question 2: Examining transcription factor binding with the WashU Epigenome Browser

To complete question 2, you will need an additional file in the Week_6 folder: 
**region_x_annotation.tsv**

This file contains the 15-state chromatin annotation for each of the 10,000 randomly-sampled genomic regions. The first column indicates the "chromosome_start_end" of the region, while the second column indicates the chromatin state, as in the chart above, such as "1_TssA". 

**Question 2a.**
The code block below implements a function to
1. sample a random genomic region from the file region_x_chrom_mark.tsv 
2. identify the chromatin state for this region from the file 15_state_chromHMM_annotations.tsv 
3. identify the cluster that this region is assigned to in your "cluster_dict" dictionary.     

In [ ]:
import numpy as np 
from numpy import random 
def sample_region(clusters,num_regions): 
    random.seed(1234)
    data=open("Week_6/region_x_chrom_mark.tsv",'r').read().strip().split('\n')
    header=data[0].split('\t')
    data=data[1::]
    annotations=open("Week_6/region_x_annotation.tsv",'r').read().strip().split('\n')
    anno_header=annotations[0].split('\t')
    annotations=annotations[1::]
    regions=[] 
    print('\t'.join(header))
    for r in range(num_regions): 
        index=random.randint(0,10000)
        cur_matrix_val=data[index]
        cur_anno=annotations[index]
        cur_cluster=clusters[index]
        entry=tuple([cur_matrix_val,cur_anno,cur_cluster])
        regions.append(entry)
        print(cur_matrix_val+'\n'+cur_anno+'\n'+str(cur_cluster)+'\n----------------------\n')
    return regions

Run the function *sample_region*  to select two random regions to examine in the WashU browser.

In [ ]:
## Uncomment the code below and replace the question mark with your code##
#regions=?

In [ ]:
print(regions)

**Question 2b.** 

Once you have obtained your regions, navigate to the browser session: 

http://epigenomegateway.wustl.edu/browser/?genome=hg19&session=8KlAnqrs4w&statusId=873262177

What transcription factors bind to each region most strongly? 

Region 1: Three strongest TF's:
1. 

2. 

3. 

Region 2: Three strongest TF's:
1. 

2. 

3. 

